Быстро строим деревья
Деревья глубины 1 обычно называют решающими пнями. Как Вы думаете, какая сложность построения решающего пня?

Представим, что мы решаем задачу регрессии, считаем для одного сплита.

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$
$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения: $\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов.

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!

Воспользуемся формулой для дисперсии:$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$Аналогично для правого сплита.

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей а) Сумму б) Сумму квадратов в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих

Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [13]:
import numpy as np

In [46]:
def get_threshold(feature, y):
    bestThreshold = None
    
    indexes = np.argsort(feature)
    sortedFeatures = feature[indexes]
    sortedY = y[indexes]
    
    sumRight = np.sum(sortedY)
    sumLeft = 0
    
    sumSquareRight = np.sum(sortedY ** 2)
    sumSquareLeft = 0
    
    amountRight = feature.shape[0]
    amountLeft = 0
    
    bestD = np.inf
    leftMean, rightMean = None, None
    for i, feature in enumerate(sortedFeatures[:-1]):
        sumRight -= y[i]
        sumLeft += y[i]
        
        sumSquareRight -= y[i] ** 2
        sumSquareLeft += y[i] ** 2
        
        amountRight -= 1
        amountLeft += 1
        
        leftD = sumSquareLeft - (sumLeft ** 2) / amountLeft
        rightD = sumSquareRight - (sumRight ** 2) / amountRight
        currentD = leftD + rightD
        
        if currentD < bestD:
            bestD = currentD
            bestThreshold = feature
            leftMean = sumLeft / amountLeft
            rightMean = sumRight / amountRight
            
    return bestD, bestThreshold, leftMean, rightMean


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = np.inf
    left_mean, right_mean = None, None
    for j in range(X.shape[1]):
        f_score, f_thr, lm, rm = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_f = j
            best_thr = f_thr
            best_error = f_score
            left_mean, right_mean = lm, rm
    return best_f, best_thr, left_mean, right_mean

In [24]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

numpy.ndarray

In [7]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

0.0
46.19909167710848


In [49]:
bestF, bestThreshold, leftMean, rightMean = train_stump(X, y)
print(bestF, bestThreshold, leftMean, rightMean, sep=' ')
yPred = []
for x in X:
    if x[bestF] > bestThreshold:
        yPred.append(rightMean)
    else:
        yPred.append(leftMean)

yPred = np.array(yPred)
print(mean_squared_error(y, yPred))

0 2.924 25.25549597855231 14.896992481202988
71.25313583527537
